In [12]:
from bs4 import BeautifulSoup

import requests

import pandas as pd

import re

from urllib.parse import urlparse

In [2]:
url_base='https://www.vghtc.gov.tw'

url_work_table='/Module/Recruitment?WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78'

url=url_base+url_work_table

g=requests.get(url)

soup=BeautifulSoup(g.content, 'html.parser')

In [3]:
soup.find('table').find_all('tr')

[<tr>
 <th><div style="text-align:left;">更新日期</div></th>
 <th>徵才類別</th>
 <th><div style="text-align:left;">徵才項目</div></th>
 <th><div style="text-align:left;">報名截止日</div></th>
 </tr>,
 <tr>
 <td data-th="更新日期">2022/11/11</td>
 <td data-th="徵才類別">住院醫師</td>
 <td class="aboutNews" data-th="徵才項目"><a href="/Module/RecruitmentContent?RecruitmentID=c124a757-10e7-48aa-9218-b6eb66bd9204&amp;WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78">皮膚科招訓112年度第一年住院醫師</a></td>
 <td data-th="報名截止日">2022/12/7</td>
 </tr>,
 <tr>
 <td data-th="更新日期">2022/11/7</td>
 <td data-th="徵才類別">醫師</td>
 <td class="aboutNews" data-th="徵才項目"><a href="/Module/RecruitmentContent?RecruitmentID=540b5743-1074-4991-8853-5a08458448de&amp;WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78">【徵才公告】泌尿外科招訓112年度第一年住院醫師甄選公告</a></td>
 <td data-th="報名截止日">2022/11/28</td>
 </tr>,
 <tr>
 <td data-th="更新日期">2022/11/7</td>
 <td data-th="徵才類別">住院醫師</td>
 <td class="aboutNews" data-th="徵才項目"><a href="/Module/RecruitmentContent?RecruitmentID=3095bb4b-

In [17]:
for tt in soup.find('table').find_all('tr'):
    for ttt in tt.findAll('td'):
        if ttt.get('data-th') == '更新日期':
            print('data',ttt.text)
        elif ttt.get('data-th') == '徵才類別':
            print('召聘職稱',ttt.text)
        elif ttt.get('data-th') == '報名截止日':
            print('期限' , ttt.text)
        elif ttt.get('data-th') == '徵才項目':
            job_detail_link = ttt.find('a')
            print('詳細連結', url_base+job_detail_link.get('href'))
        else:
            print(ttt)


data 2022/11/11
召聘職稱 住院醫師
詳細連結 https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=c124a757-10e7-48aa-9218-b6eb66bd9204&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78
期限 2022/12/7
data 2022/11/7
召聘職稱 醫師
詳細連結 https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=540b5743-1074-4991-8853-5a08458448de&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78
期限 2022/11/28
data 2022/11/7
召聘職稱 住院醫師
詳細連結 https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=3095bb4b-cd93-4efe-aa6a-c0371b3f18d4&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78
期限 2022/11/25
data 2022/11/3
召聘職稱 技術人員
詳細連結 https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=4b8fee47-de8e-492d-983f-9ed4568c0f16&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78
期限 2022/11/14
data 2022/11/1
召聘職稱 護理人員
詳細連結 https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=44f9e8a7-a884-4c83-8879-889a53ed2d7b&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78
期限 2022/11/19
data 2022/11/1
召聘職稱 技術人員
詳細連結 https:/

In [26]:
work_table=[]
for tt in soup.find('table').find_all('tr'):
    _one_job_data ={}
    for ttt in tt.findAll('td'):
        
        if ttt.get('data-th') == '更新日期':
            #print('data',ttt.text)
            pass
        elif ttt.get('data-th') == '報名截止日':
            #print('期限' , ttt.text)
            _one_job_data['deadline'] = ttt.text
        elif ttt.get('data-th') == '徵才項目':
            _one_job_data['jobtype'] = ttt.text
            try:
                _one_job_data['job_type'] = re.search(r"[胸,皮,泌,眼,內,外,藥,麻,臨,急]\w+[科,部]",ttt.text.replace('【徵才公告】','')).group(0)
            except:
                print(ttt.text,'error')
            job_detail_link = url_base+ttt.find('a').get('href')
            #print('詳細連結', job_detail_link)
            _one_job_data['job_detail_link'] = job_detail_link
            g=requests.get(job_detail_link)
            _soup=BeautifulSoup(g.content, 'html.parser')
            try:
                job_file_link = url_base+_soup.find('a',_class="btn-recruit-add").get('href')
            except:
                job_file_link = None
            _one_job_data['job_file_link'] = job_file_link
    print(_one_job_data)
    if _one_job_data != {}:
        work_table.append(_one_job_data)
    print(pd.DataFrame(work_table))

{}
Empty DataFrame
Columns: []
Index: []
{'jobtype': '皮膚科招訓112年度第一年住院醫師', 'job_type': '皮膚科', 'job_detail_link': 'https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=c124a757-10e7-48aa-9218-b6eb66bd9204&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78', 'job_file_link': None, 'deadline': '2022/12/7'}
             jobtype job_type  \
0  皮膚科招訓112年度第一年住院醫師      皮膚科   

                                     job_detail_link job_file_link   deadline  
0  https://www.vghtc.gov.tw/Module/RecruitmentCon...          None  2022/12/7  
{'jobtype': '【徵才公告】泌尿外科招訓112年度第一年住院醫師甄選公告', 'job_type': '泌尿外科', 'job_detail_link': 'https://www.vghtc.gov.tw/Module/RecruitmentContent?RecruitmentID=540b5743-1074-4991-8853-5a08458448de&WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78', 'job_file_link': None, 'deadline': '2022/11/28'}
                        jobtype job_type  \
0             皮膚科招訓112年度第一年住院醫師      皮膚科   
1  【徵才公告】泌尿外科招訓112年度第一年住院醫師甄選公告     泌尿外科   

                                     job_det

# Get page link
In this website the work table has some page, so the function should find page link for next step.

In [4]:
pages_link = []

pages_link.append(url)

for pp in soup.find('div', class_="pager").find_all('a'):
    if pp.get('title') == 'Next page':
        pass
    else:
        pages_link.append(url_base+pp.get('href'))
pages_link

['https://www.vghtc.gov.tw/Module/Recruitment?WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78',
 'https://www.vghtc.gov.tw/Module/Recruitment?WebMenuID=9d005e46-411b-46fc-b438-e0bd561eba78&page=2']

# Get work table

The get_work_page function is used for getting each work page
